In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
ls


gdrive/  sample_data/


In [3]:
cd gdrive/My\ Drive/Colab\ Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [4]:
ls


'Assignment#1_Face Recognition.ipynb'               Micrpsoft_Test.ipynb
'Copy of Assignment#1_Face Recognition (1).ipynb'   Project#3_PR.ipynb
'Copy of Assignment#1_Face Recognition.ipynb'       RML2016.10b.dat
'Copy of Offensive_tweets.ipynb'                    RML2016.10b.tar.bz2
'Copy of Sheet#1_PR.ipynb'                          RML2016.10b.tar.bz2.1
'Copy of Sheet#2_PR.ipynb'                          RML2016.10b.tar.bz2.2
'Copy of Sheet#4_PR.ipynb'                          RML2016.10b.tar.bz2.3
 Data2.npy                                          RML2016.10b.tar.bz2.4
 Data3.npy                                          RML2016.10b.tar.bz2.5
 Data_diff.npy                                      Sheet#1_Networks.ipynb
 Data_integ.npy                                     Sheet#2_PR.ipynb
 LICENSE.TXT


In [6]:
!wget  http://opendata.deepsig.io/datasets/2016.10/RML2016.10b.tar.bz2 

--2019-05-03 17:02:44--  http://opendata.deepsig.io/datasets/2016.10/RML2016.10b.tar.bz2
Resolving opendata.deepsig.io (opendata.deepsig.io)... 52.14.91.165
Connecting to opendata.deepsig.io (opendata.deepsig.io)|52.14.91.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1245608913 (1.2G) [application/x-bzip2]
Saving to: ‘RML2016.10b.tar.bz2.5’

RML2016.10b.tar.bz2 100%[===================>]   1.16G  10.0MB/s    in 1m 52s  

2019-05-03 17:04:36 (10.6 MB/s) - ‘RML2016.10b.tar.bz2.5’ saved [1245608913/1245608913]



In [0]:
!tar xjf "RML2016.10b.tar.bz2"

In [5]:
import os,random
import numpy as np
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
from keras.optimizers import adam
import matplotlib.pyplot as plt
import seaborn as sns
import pickle, random, sys, keras
from keras.models import Sequential
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [0]:
DataLoaded = pickle.load(open("RML2016.10b.dat",'rb'))
Inphases,Quadratures = map(lambda j: sorted(list(set(map(lambda x: x[j], DataLoaded.keys())))), [1,0])
Data = []  
Labels = []
for Quadrature in Quadratures:
    for Inphase in Inphases:
#       data index with this modulation and snr
        Data.append(DataLoaded[(Quadrature,Inphase)])
#         append labels
        for i in range(DataLoaded[(Quadrature,Inphase)].shape[0]):  Labels.append((Quadrature,Inphase))
Data = np.vstack(Data)

In [10]:
print(Data.shape)

(1200000, 2, 128)


In [11]:
# modulation type and snr
print(Labels[0]) 


('8PSK', -20)


In [0]:
# Data_diff = np.apply_along_axis(np.gradient, 1, Data)

In [0]:
# print(Data_diff.shape)

In [0]:
# from scipy.integrate import cumtrapz

# Data_integ = np.apply_along_axis(cumtrapz, 1, Data,initial=0)

In [0]:
# print(Data_integ.shape)

In [0]:
# np.save('Data_integ', Data_integ)
Data_integ = np.load('Data_integ.npy')

In [0]:
# np.save('Data_diff', Data_diff)
Data_diff = np.load('Data_diff.npy')

In [11]:
print(Data_diff.shape)

(1200000, 2, 128)


In [0]:
# # combining data with the differentiation
# Data2 = []
# for i in range(0,1200000):
#    Data2.append(np.vstack((Data[i],Data_diff[i])))
    


In [0]:
# Data2=np.asarray(Data2)
# print(Data2.shape)

In [0]:
# np.save('Data2', Data2)
Data2 = np.load('Data2.npy')

In [0]:
# combining data with the differentiation with the integration
Data3 = []
for i in range(0,1200000):
   Data3.append(np.vstack((Data2[i],Data_integ[i])))
    


In [24]:
Data3=np.asarray(Data3)
print(Data3.shape)

(1200000, 6, 128)


In [14]:
%reset


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [0]:
# np.save('Data3', Data3)
Data3 = np.load('Data3.npy')

# Split the data into 50% for training/validation and 50% for testing. 

In [0]:
from sklearn import preprocessing

LB = np.asarray(Labels)[:,0]
x = preprocessing.LabelBinarizer()
x.fit(LB)
LB = x.transform(LB)

In [0]:
train_vectors = np.array(Data3)[list(range(0,1200000,2)),:]
train_labels = np.array(LB)[list(range(0,1200000,2))]

test_vectors = np.array(Data3)[list(range(1,1200000,2)),:]
test_labels = np.array(LB)[list(range(1,1200000,2))]




In [0]:
print(train_vectors.shape)

# Fully Connected Neural Net as a baseline

In [0]:
# # use ReLU
# model = Sequential([
#     Dense(32, input_shape=(2, 128)), 
#     Activation('relu'),
#     Dense(10),
#     Activation('softmax'),
# ])

In [0]:
model = Sequential()
model.add(Reshape((256,), input_shape=(2, 128)))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [0]:
# compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
# fit
model.fit(train_vectors, train_labels, epochs=150, batch_size=1024, validation_split=0.05, verbose = 2, callbacks=[EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode="min")])


Train on 570000 samples, validate on 30000 samples
Epoch 1/150
 - 3s - loss: 1.3040 - acc: 0.4926 - val_loss: 13.0683 - val_acc: 0.0194
Epoch 2/150
 - 3s - loss: 1.3033 - acc: 0.4928 - val_loss: 13.1792 - val_acc: 0.0181
Epoch 3/150
 - 3s - loss: 1.3037 - acc: 0.4931 - val_loss: 12.5618 - val_acc: 0.0272
Epoch 4/150
 - 3s - loss: 1.3023 - acc: 0.4930 - val_loss: 13.1335 - val_acc: 0.0184
Epoch 5/150
 - 3s - loss: 1.3007 - acc: 0.4935 - val_loss: 12.9330 - val_acc: 0.0218
Epoch 6/150
 - 3s - loss: 1.3003 - acc: 0.4940 - val_loss: 13.3048 - val_acc: 0.0163
Epoch 7/150
 - 3s - loss: 1.2999 - acc: 0.4945 - val_loss: 13.2102 - val_acc: 0.0176
Epoch 8/150
 - 3s - loss: 1.2987 - acc: 0.4949 - val_loss: 13.0533 - val_acc: 0.0215
Epoch 9/150
 - 3s - loss: 1.2976 - acc: 0.4952 - val_loss: 13.1485 - val_acc: 0.0207
Epoch 10/150
 - 3s - loss: 1.2972 - acc: 0.4953 - val_loss: 13.2864 - val_acc: 0.0172
Epoch 11/150
 - 3s - loss: 1.2957 - acc: 0.4961 - val_loss: 13.4995 - val_acc: 0.0135
Epoch 12/150

In [48]:
# evaluate

score = model.evaluate(test_vectors, test_labels, verbose=0, batch_size=1024)
print score

[2.1252029591248434, 0.39598833333342026]


# CNN

In [0]:
# # use ReLU
# model_CNN = Sequential([
#     Dense(32, input_shape=(2, 128)), 
#     Activation('relu'),
#     Dense(10),
#     Activation('softmax'),
# ])

In [0]:
model_CNN = Sequential()
model_CNN.add(Reshape((128,2,1), input_shape=(2,128)))
model_CNN.add(Conv2D(64, (1, 3), activation='relu', padding="same"))
model_CNN.add(Dropout(0.5))
model_CNN.add(Conv2D(16, (2,3), activation='relu', padding="same"))
model_CNN.add(Dropout(0.5))
model_CNN.add(Flatten())
model_CNN.add(Dense(128, activation='relu'))
model_CNN.add(Dense(10, activation='softmax'))


In [0]:
# compile
model_CNN.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
# fit
model_CNN.fit(train_vectors, train_labels, epochs=150, batch_size=1024, validation_split=0.05, verbose = 2, callbacks=[EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode="min")])


Train on 570000 samples, validate on 30000 samples
Epoch 1/150
 - 19s - loss: 2.1376 - acc: 0.1763 - val_loss: 3.0257 - val_acc: 0.0011
Epoch 2/150
 - 16s - loss: 1.8616 - acc: 0.2797 - val_loss: 3.8797 - val_acc: 7.3333e-04
Epoch 3/150
 - 16s - loss: 1.7596 - acc: 0.3235 - val_loss: 3.9641 - val_acc: 8.6667e-04
Epoch 4/150
 - 16s - loss: 1.6937 - acc: 0.3473 - val_loss: 4.2871 - val_acc: 1.0000e-03
Epoch 5/150
 - 16s - loss: 1.6550 - acc: 0.3591 - val_loss: 4.6324 - val_acc: 1.0000e-03
Epoch 6/150
 - 15s - loss: 1.6373 - acc: 0.3648 - val_loss: 4.7236 - val_acc: 0.0021
Epoch 7/150
 - 16s - loss: 1.6256 - acc: 0.3681 - val_loss: 4.8337 - val_acc: 0.0020
Epoch 8/150
 - 16s - loss: 1.6164 - acc: 0.3715 - val_loss: 5.2690 - val_acc: 0.0024
Epoch 9/150
 - 16s - loss: 1.6084 - acc: 0.3730 - val_loss: 4.9609 - val_acc: 0.0052
Epoch 10/150
 - 15s - loss: 1.5991 - acc: 0.3759 - val_loss: 5.3883 - val_acc: 0.0036
Epoch 11/150
 - 16s - loss: 1.5917 - acc: 0.3782 - val_loss: 5.2836 - val_acc: 0.0

In [59]:
# evaluate

score = model_CNN.evaluate(test_vectors, test_labels, verbose=0, batch_size=1024)
print score

[1.737546489683787, 0.3732833333335817]
